# Prepare datasets for publishing

In [ ]:
%matplotlib inline

import sys
import warnings
import odc.geo.xr
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
from odc.geo.xr import assign_crs

## Open datasets


In [ ]:
p_average = assign_crs(xr.open_dataset(f'/g/data/os22/chad_tmp/Aus_phenology/results/mean_phenology_perpixel_circular.nc'), crs='EPSG:4326')
p_trends = assign_crs(xr.open_dataset(f'/g/data/os22/chad_tmp/Aus_phenology/results/trends_phenology_perpixel_circular.nc'), crs='EPSG:4326')

## Masking

In [ ]:
season_per_year = p_average['n_seasons']/p_average['n_years']
non_seasonal = xr.where((season_per_year <= 0.90),1,0)
extra_seasonal = xr.where((season_per_year >= 1.1),1,0)
seasonality_mask = (non_seasonal | extra_seasonal)

sos_nan = np.isnan(p_trends['SOS_slope']).where(~np.isnan(p_average['SOS']))
sos_nan = xr.where((sos_nan==1) & (seasonality_mask==0), 1, 0)

eos_nan = np.isnan(p_trends['EOS_slope']).where(~np.isnan(p_average['EOS']))
eos_nan = xr.where((eos_nan==1) & (seasonality_mask==0), 1, 0)

pos_nan = np.isnan(p_trends['POS_slope']).where(~np.isnan(p_average['POS']))
pos_nan = xr.where((pos_nan==1) & (seasonality_mask==0), 1, 0)

extra_mask = (seasonality_mask | sos_nan | eos_nan | pos_nan)

In [ ]:
p_average = p_average.where(seasonality_mask!=1)
p_trends = p_trends.where(extra_mask!=1)

## Plot

In [ ]:
fig,ax = plt.subplots(1,2, figsize=(10,4),sharey=True, layout='constrained')
p_average['SOS'].plot(ax=ax[0], cmap='twilight')
p_trends['SOS_slope'].plot(ax=ax[1], cmap='RdBu_r');
ax[0].set_title('SOS')
ax[1].set_title('SOS trend');

## Add metadata

In [ ]:
attrs_dict_average={}
attrs_dict_average['nodata'] = np.nan
attrs_dict_average['crs'] = 'EPSG:4326'
attrs_dict_average['version'] = 'v1.0'
attrs_dict_average['description'] = 'Long-term (1982-2022) average land surface phenology for Australia. Variables SOS, POS, EOS, TOS are calculated using the circular mean approach, while the SOS_std, POS_std, EOS_std, TOS_std refer to the circular standard deviation of the same variables. All other variables are calculated using a median statistic. Refer to the accompanying publication for acronym descriptions.'
attrs_dict_average['citation'] = 'TBC'
attrs_dict_average['email'] = 'chad.burton@anu.edu.au'
p_average.attrs = attrs_dict_average
p_average

In [ ]:
attrs_dict_trend={}
attrs_dict_trend['nodata'] = np.nan
attrs_dict_trend['crs'] = 'EPSG:4326'
attrs_dict_trend['version'] = 'v1.0'
attrs_dict_trend['description'] = 'Long-term (1982-2022) land surface phenology trends for Australia. Refer to the accompanying publication for acronym  descriptions and for a description of how trends are calculated.'
attrs_dict_trend['citation'] = 'TBC'
attrs_dict_trend['email'] = 'chad.burton@anu.edu.au'
p_trends.attrs = attrs_dict_trend
p_trends

## Export

In [ ]:
p_average.to_netcdf('/g/data/os22/chad_tmp/Aus_phenology/results/publication_datasets/Australian_LSP_average_1982_2022_v1.0.nc')
p_trends.to_netcdf('/g/data/os22/chad_tmp/Aus_phenology/results/publication_datasets/Australian_LSP_trends_1982_2022_v1.0.nc')